# Step 3: gather asset metadata - UC Davis Campus Energy

**GraphQL -> python module**

In [ ]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import urllib3

# Neo4J graph database
import py2neo 
from py2neo import Graph
from py2neo import Node, Relationship

import yaml

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [ ]:
# config_file = "config-windfarm.yaml"  # "config-acad-prod-desc-v2.yaml"
config_file = "config-prod-ucd-v2.yaml"

In [ ]:
with open(config_file) as f:
    config = yaml.safe_load(f)
config

In [ ]:
# dv = build_dv("FV31", "all_columns", "Default")
# print(json.dumps(dv, indent=2))

In [ ]:
graph = Graph(
    config["neo4j"]["host"],
    auth=(config["neo4j"]["user"], config["neo4j"]["password"]),
    name=config["neo4j"]["database"],
)

In [ ]:
sample_transport = RequestsHTTPTransport(
    url="https://data.academic.osisoft.com/hubgraphql", verify=False, retries=3
)
client = Client(transport=sample_transport, fetch_schema_from_transport=True)

In [ ]:
meta_query = gql(
    """
{
  Database(name: "Campus_Energy") {
    asset_with_dv(orderBy: name_asc) {
      name
      id
      has_attribute(filter: { hub_meta: true }, orderBy: name_asc) {
        name
        value
      }
      has_element(
        orderBy: name_asc
        filter: {
          OR: [
            { name: "Electricity" }
            { name: "ChilledWater" }
            { name: "Steam" }
          ]
        }
      ) {
        name
        has_attribute(filter: { hub_meta: true }, orderBy: name_asc) {
          name
          value
        }
      }
    }
  }
}
"""
)
result = client.execute(meta_query)

In [ ]:
result

In [ ]:
result["Database"][0]["asset_with_dv"][0]

In [ ]:
def convert2value(s):
    if "." in s:
        try:
            return(float(s))
        except:
            pass
    try:
        return(int(s))
    except:
        return s

In [ ]:
convert2value("    1.")

In [ ]:
def extract_meta(js, sub=False):
    j = js["has_attribute"]
    prefix = "" if not sub else (js["name"].lower() + ".")
    return {f"{prefix}{k['name']}": convert2value(k["value"]) for k in j}

In [ ]:
d = extract_meta(result["Database"][0]["asset_with_dv"][0])
d

In [ ]:
d.update(extract_meta(result["Database"][0]["asset_with_dv"][0]['has_element'][0], True))

In [ ]:
len(result["Database"][0]["asset_with_dv"])

In [ ]:
meta_mutation = gql(
    """
mutation UpdateMeta($id: ID!, $meta: String) {
  MergeElement(id: $id, hub_metadata: $meta) {
    id
    name
    hub_metadata
  }
}
"""
)

element_query = gql(
    """
query ElementById($id: ID!) { 
  Element(id: $id) {
    id
    af_template
    asset_db
    name
  }
}
"""
)


def update_meta(wid, meta):
    result = client.execute(
        meta_mutation, variable_values={"id": wid, "meta": str(meta)}
    )
    return result


def element_by_id(wid):
    result = client.execute(element_query, variable_values={"id": wid})
    return result

In [ ]:
element_by_id("F1EmzO41BZK4DE21ZU4Yk9YqRQHcG0pq6i5xGpRwANOjB-OwVU5JLVBJQUYtVk0wXFVDIERBVklTXFVDIERBVklTXEJVSUxESU5HU1xBUkMgUEFWSUxJT04")

In [ ]:
nb = len(result["Database"][0]["asset_with_dv"])
for i in range(nb):
    dai = result["Database"][0]["asset_with_dv"][i]
    d = extract_meta(dai)
    for j in range(len(dai["has_element"])):
        d.update(extract_meta(dai["has_element"][j], True))
    r = element_by_id(dai["id"])
    print(dai["name"], dai["id"], d)
    # update_meta(dai["id"], d, r["Element"][0])
    gr = graph.run(
        f"match (e:Element) where e.id='{dai['id']}' set e.hub_metadata=\"{str(d)}\" return e"
    ).data()